### Code to retrieve GEE dataset to specified region and for specified resolution

- used for testing the SRTM and MERIT-HYDRO data that is available on Google Earth Engine

In [5]:
import xarray as xr

# Earth engine imports
import ee
import folium
import geehydro

ee.Authenticate()
ee.Initialize()


Enter verification code: 4/1AWgavdeAxiFRjbkclQiZi1SBHW0C-spo6qwdz0Zhb3UqF-FuxwhurcfxEtg

Successfully saved authorization token.


In [7]:
# reading in an existing dataset that has the required image extents (alternatively just enter them in cell below)
#ALOS_OUT = add_filepath

pw_corrs = xr.open_dataset(ALOS_OUT + 'corrs_da_HH_new.nc')

In [8]:
# ALOS-2 image extents
# making the area slightly larger so that the interpolation to a lower resolution doesn't cut out the border data
lat_min = pw_corrs.lat.min().values.item() - 0.01
lat_max = pw_corrs.lat.max().values.item() + 0.01
lon_min = pw_corrs.lon.min().values.item() - 0.01
lon_max = pw_corrs.lon.max().values.item() + 0.01

print (lat_min,lat_max)
print (lon_min,lon_max)

-1.0906201868375438 2.768779337489291
16.10553549932626 19.910137791321805


In [17]:
# GEE dataset location
image_address = 'CSP/ERGo/1_0/Global/SRTM_landforms'

# bands to be retrieved
bands = ['constant']

# resolution (original, or to be interpolated to)
resn = 90

# image extents
extents = [lon_min, lat_min, lon_max, lat_max]

# description of the dataset (the filename it will be saved to)
description = 'SRTM_landform'

In [16]:
def retrieve_gee_data(image_address, bands, resn, extents,description):
    # retrieve GEE data
    bands_final = ee.Image(image_address).select(bands)
    roi = ee.Geometry.Rectangle(extents);

    # extracting the data
    data = bands_final.reduceRegion(reducer = ee.Reducer.mean(), geometry = roi, scale = resn)

    #print (data)

    # downloading as a geotiff file
    task = ee.batch.Export.image.toDrive(\
                                         image = bands_final,\
                                         description = description,\
                                         fileNamePrefix = description,\
                                         region = roi,\
                                         scale = resn,\
                                         maxPixels = 1e10\
                                        )

    # starting the task to download the data
    task.start()
    
    # monioring task status
    print (task.status())   
    
    return (task)

In [18]:
retrieve_gee_data(image_address, bands, resn, extents, description)

{'state': 'READY', 'description': 'SRTM_landform', 'creation_timestamp_ms': 1673634245322, 'update_timestamp_ms': 1673634245322, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JY7GGEGQTSAN3I6NHBIRWZ4M', 'name': 'projects/earthengine-legacy/operations/JY7GGEGQTSAN3I6NHBIRWZ4M'}


In [ ]:
print (task.status())